In [1]:
"""
    INITIALIZE Hintereisferner:
    Run all necessary preprocessing tasks up to the mu* calibration
    (i.e. `local_mu_star`). File -> Make a copy... in order to use as
    template for further experiments.

    Date: 07.02.2019
"""
# import externals libs
import os
import shutil
import numpy as np
import pandas as pd
import geopandas as gpd

# import the needed OGGM modules
import oggm
from oggm import cfg
from oggm.utils import get_demo_file, get_rgi_glacier_entities
from oggm.tests.funcs import get_test_dir
from oggm.core import gis, climate, centerlines, vascaling

# ---------------------
#  PREPROCESSING TASKS
# ---------------------

# create test directory
testdir = os.path.join(get_test_dir(), 'tmp_comparison')
if not os.path.exists(testdir):
    os.makedirs(testdir)
shutil.rmtree(testdir)
os.makedirs(testdir)

# load default parameter file
cfg.initialize()
cfg.PATHS['working_dir'] = testdir

cfg.PATHS['dem_file'] = get_demo_file('hef_srtm.tif')
cfg.PARAMS['border'] = 10

cfg.PARAMS['baseline_climate'] = 'HISTALP'
cfg.PARAMS['prcp_scaling_factor'] = 1.75
cfg.PARAMS['temp_melt'] = -1.75

cfg.PARAMS['use_multiprocessing'] = True

cfg.set_intersects_db(get_demo_file('rgi_intersect_oetztal.shp'))

# read the Hintereisferner DEM
hef_file = get_demo_file('Hintereisferner_RGI5.shp')
entity = gpd.read_file(hef_file).iloc[0]

# initialize the GlacierDirectory
gdir = oggm.GlacierDirectory(entity, base_dir=testdir)
# define the local grid and glacier mask
gis.define_glacier_region(gdir, entity=entity)
gis.glacier_masks(gdir)

2019-05-27 13:06:06: oggm.cfg: Using configuration file: /Users/oberrauch/oggm-fork/oggm/params.cfg
2019-05-27 13:06:06: oggm.core.gis: (RGI50-11.00897) define_glacier_region
2019-05-27 13:06:06: oggm.core.gis: (RGI50-11.00897) glacier_masks


In [2]:
# process the given climate file
climate.process_histalp_data(gdir)

2019-05-27 13:06:07: oggm.core.climate: (RGI50-11.00897) process_histalp_data


In [6]:
# run center line preprocessing tasks
centerlines.compute_centerlines(gdir)
centerlines.initialize_flowlines(gdir)
centerlines.compute_downstream_line(gdir)
centerlines.compute_downstream_bedshape(gdir)
centerlines.catchment_area(gdir)
centerlines.catchment_intersections(gdir)
centerlines.catchment_width_geom(gdir)
centerlines.catchment_width_correction(gdir)

2019-02-25 11:24:55: oggm.core.centerlines: (RGI50-11.00897) compute_centerlines
2019-02-25 11:24:55: oggm.core.centerlines: (RGI50-11.00897) initialize_flowlines
2019-02-25 11:24:55: oggm.core.centerlines: (RGI50-11.00897) compute_downstream_line
2019-02-25 11:24:56: oggm.core.centerlines: (RGI50-11.00897) compute_downstream_bedshape
2019-02-25 11:24:56: oggm.core.centerlines: (RGI50-11.00897) catchment_area
2019-02-25 11:24:58: oggm.core.centerlines: (RGI50-11.00897) catchment_intersections
2019-02-25 11:24:58: oggm.core.centerlines: (RGI50-11.00897) catchment_width_geom
2019-02-25 11:24:59: oggm.core.centerlines: (RGI50-11.00897) catchment_width_correction


In [4]:
gdir.read_pickle('climate_info')

{'baseline_climate_source': 'HISTALP',
 'baseline_hydro_yr_0': 1802,
 'baseline_hydro_yr_1': 2014}

In [7]:
from oggm.utils import ncDataset

In [14]:
histalp_nc = ncDataset(oggm.utils.get_demo_file('histalp_merged_hef.nc'))
histalp_nc.variables

OrderedDict([('time', <class 'netCDF4._netCDF4.Variable'>
              int32 time(time)
                  units: days since 1801-01-01 00:00:00
              unlimited dimensions: time
              current shape = (2424,)
              filling on, default _FillValue of -2147483647 used),
             ('lat', <class 'netCDF4._netCDF4.Variable'>
              float64 lat(lat)
                  long_name: latitude
                  units: degrees_north
                  standard_name: latitude
              unlimited dimensions: 
              current shape = (3,)
              filling on, default _FillValue of 9.969209968386869e+36 used),
             ('lon', <class 'netCDF4._netCDF4.Variable'>
              float64 lon(lon)
                  long_name: longitude
                  units: degrees_east
                  standard_name: longitude
              unlimited dimensions: 
              current shape = (3,)
              filling on, default _FillValue of 9.969209968386869e+36 use

In [12]:
histalp_nc

<class 'netCDF4._netCDF4.Variable'>
float32 temp(time, lat, lon)
    long_name: 2m temperature
    units: degC
unlimited dimensions: time
current shape = (2424, 3, 3)
filling on, default _FillValue of 9.969209968386869e+36 used

In [6]:
oggm.utils.get_demo_file('histalp_merged_hef.nc')

'/Users/oberrauch/.oggm/oggm-sample-data-1e67ee089d58a8171342b948a68b4cf09a8d948c/test-files/histalp_merged_hef.nc'